In [3]:
import sys 
import csv
import os
from datetime import datetime
from datetime import timedelta

########Parameters Setting############
len_day = 182
#year = '2016'
#start_month = '3'
sentiment_file = 'acc_sentiment_2015_11to2016_04_without_filter.csv'
######################################

#Retrieve stock price for a particular day
def retrieveStockPrice(company):
    stock_file = os.path.join("SandP500_stock", company + "_data.csv")
    with open(stock_file) as stockfile:
        stockprice = {}
        reader = csv.DictReader(stockfile)
        for row in reader:
            if 'open' in row:
                if row["open"] and row['close']:
                    stockprice[row['date']] = round((float(row["open"]) + float(row['close'])) / 2, 5)
            else:
                if row["Open"] and row['Close']:
                    stockprice[row['Date']] = round((float(row["Open"]) + float(row['Close'])) / 2, 5)
    return stockprice

def transformwithSlash(date):
    if date.month < 10:
        date_string_month = str(date.year) + '-0' + str(date.month)                    
    else:
        date_string_month = str(date.year) + '-' + str(date.month)
    
    if date.day < 10:
        date_string = str(date_string_month) +  '-0' + str(date.day)
    else:
        date_string = str(date_string_month) +  '-' + str(date.day)       
    return date_string

#Retriev accumulated sentiment and compare with actual stock price
company_list = []
with open('SP500.csv') as SP500csv:
    reader = csv.DictReader(SP500csv)
    for row in reader:
        company_list.append(row['Symbol'])        
n_company = len(company_list)
ROI_history = [0] * n_company
for i in range(n_company):
    ROI_history[i] = [0] * len_day
    
currHold = 0
with open(sentiment_file) as accSentiment:
    reader = csv.DictReader(accSentiment)
    company_idx = 0
    for acc_Sentiment in reader:
        currHold = 0
        company = company_list[company_idx]
        avg_price = retrieveStockPrice(company)
                
        sumSentiment = 0
        day = -1
        #for day in range(1, len_day):
        for today in acc_Sentiment:
            day+=1
            date = datetime.strptime(today,'%m/%d/%Y')
            
            next_date = date + timedelta(days=1)  #Buy stock 1 day after
            #date =  str(int(month)) + '/' + str(day) + '/' + year
            sumSentiment += int(acc_Sentiment[today])          
            
            if day==0:   
                #Find first workday in the month.
                findworkday = date
                findwd_string = transformwithSlash(findworkday)
                while findwd_string not in avg_price:
                    findworkday += timedelta(days=1)
                    findwd_string = transformwithSlash(findworkday)
                previousPrice = avg_price[findwd_string]
                
                #Uncomment if buy stock at the begining.
                currHold = 100 /  previousPrice
            
            #if day <= 31:
            #    date = '5/' + str(day) + '/2016'
            #    month = 5
            #    next_date = datetime(2016, month, day) + timedelta(days=1)  #Buy stock 1 day after
            #else:
            #    date = '5/' + str(day-30) + '/2016'
            #    month = 5
            #    next_date = datetime(2016, month, day-30) + timedelta(days=1)  #Buy stock 1 day after
            #sumSentiment += int(acc_Sentiment[date])
            
            #next_date = datetime(2016, month, day) + timedelta(days=1)  #Buy stock 1 day after
            
            next_date_string = transformwithSlash(next_date)
                        
            if next_date_string in avg_price:
                nextdayPrice = avg_price[next_date_string]                
                if sumSentiment > 0:
                    if currHold == 0:
                        currHold = 100 / nextdayPrice
                        #bgtPrice = avg_price[t_string]
                        previousPrice = avg_price[next_date_string]
                elif sumSentiment < 0:
                    if currHold:
                        currHold = 0
                        
                if currHold:       
                    ROI_history[company_idx][day] = round(ROI_history[company_idx][day-1] + (nextdayPrice-previousPrice)*currHold , 5)
                else:
                    ROI_history[company_idx][day] = round(ROI_history[company_idx][day-1], 5)

                previousPrice = nextdayPrice #Save stock price for next iteration
                sumSentiment = 0 #Reset sentiment for every open market day
            else:
                ROI_history[company_idx][day] = round(ROI_history[company_idx][day-1], 5)
                

                            
        company_idx+=1 #Predict next stock
        
        

In [4]:
import pprint
import numpy as np
arr = np.array(ROI_history)
#com = np.array(company_list)[np.newaxis]
#new_arr = np.hstack([com.T, arr])
np.savetxt('ROI_history_2015_11to2016_04_without_filter_buyall.csv', arr, delimiter=',', fmt='%1.5f')

#Generate dates
dates = []
#for thedate in range(1, 32):
    #dates.append('12/'+ str(thedate) + '/2015')
for thedate in range(1, 31):
    dates.append('11/'+ str(thedate) + '/2015')
for thedate in range(1, 32):
    dates.append('12/'+ str(thedate) + '/2015')
for thedate in range(1, 32):
    dates.append('1/'+ str(thedate) + '/2016')
for thedate in range(1, 30):
    dates.append('2/'+ str(thedate) + '/2016')
for thedate in range(1, 32):
    dates.append('3/'+ str(thedate) + '/2016')
for thedate in range(1, 31):
    dates.append('4/'+ str(thedate) + '/2016')

roipath = 'ROI_history_2015_11to2016_04_without_filter_buyall.csv'
with open(roipath) as outcsv:
    r = csv.reader(outcsv)
    data = [line for line in r]
with open(roipath, 'w', newline='') as outcsv:
    writer = csv.writer(outcsv)
    writer.writerow(dates)
    writer.writerows(data)

In [60]:
def savestockprice(company, year, start_month, end_month):
    price = retrieveStockPrice(company)
    date = datetime(year,start_month,1)
    if end_month == 12:
        end_date = datetime(year+1,1,1)
    else:
        end_date = datetime(year,end_month+1,1)
        
    len_day = (end_date - date).days
    date_string = [0] * len_day
    daily_price = [0] * len_day
    for day in range(len_day):
        cur_date =  transformwithSlash(date + timedelta(days=day))
        date_string[day] = cur_date
        if cur_date in price:
            daily_price[day] = price[cur_date]
        else:
            daily_price[day] = daily_price[day-1]
    
    with open(company+'_'+str(year)+'_'+str(start_month)+'to'+str(end_month)+'.csv', 'w', newline='') as outcsv:
        writer = csv.writer(outcsv)
        writer.writerow(date_string)
        writer.writerow(daily_price)


savestockprice('SPY', 2016, 1, 4)            
    